# EMISSOR chat bot with a BRAIN that interacts through camera and NER

@CLTL

## 1. Introduction

In this demo, we create capsules with triples from interaction through EMISSOR that are send to the Brain.

This notebook demonstrates a simple chatbot that also takes pictures through your camera while interacting with you

## 2. Installing the platform

virtualenv venv

# install the brain
#pip install --extra-index-url https://test.pypi.org/simple cltl.brain

# install the Leolani baseline triple extractor
#pip install git+git://github.com/leolani/cltl-knowledgeextraction.git@main

# install emissor
#%python3 -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple emissor

Alternatively:
* git clone https://github.com/cltl/EMISSOR --branch issue-53-processing
* cd EMISSOR
* python install.py install

# install spacy
#%pip install -U spacy

# install language models
#%python -m spacy download en_core_web_sm

# install cv2
#%pip install opencv-python

In [1]:
!pip install git+git://github.com/leolani/cltl-knowledgeextraction.git@main

  Cloning git://github.com/leolani/cltl-knowledgeextraction.git (to revision main) to /private/var/folders/6w/bw7dqbl9727c48pcjjh32r140000gn/T/pip-req-build-dps1yd1a
  Running command git clone -q git://github.com/leolani/cltl-knowledgeextraction.git /private/var/folders/6w/bw7dqbl9727c48pcjjh32r140000gn/T/pip-req-build-dps1yd1a
  Resolved git://github.com/leolani/cltl-knowledgeextraction.git to commit e07bd458cdd61c5cea4e075faee9798c8227eb7c
  Created wheel for cltl.triple-extraction: filename=cltl.triple_extraction-0.0.dev3-py3-none-any.whl size=70736753 sha256=da00cdda632ac9213fe9dddbfeea248851a91eda3d79e61a94ac056ebb18ba99
  Stored in directory: /private/var/folders/6w/bw7dqbl9727c48pcjjh32r140000gn/T/pip-ephem-wheel-cache-4hyp9wll/wheels/15/c1/aa/628ed9d1c391e8f80121d46d441a472ae5c6491e7edf80d37a
Successfully built cltl.triple-extraction
You should consider upgrading via the '/Users/piek/PycharmProjects/cltl-chatbots/venv/bin/python -m pip install --upgrade pip' command.


## 3. Setting the scene

In [2]:
import emissor
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis

In [ ]:
import os
import spacy
import time
import uuid
from datetime import datetime
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
import cv2
from datetime import datetime
from datetime import date
from random import getrandbits
import requests

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import dummies.text_to_triple as ttt

### 3.1 Creating the scenario and basic values

In [5]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
agent = "Leolani2"
human = "Stranger"

### The name of your scenario
scenarioid = "myscenario3"

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "./data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenarioid + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenarioid)
scenario = scenarioStorage.create_scenario(scenarioid, datetime.now().microsecond, datetime.now().microsecond, agent)

Directory  ./data/myscenario3  Created 
Directory  ./data/myscenario3/image  Created 


### 3.2 Adding NERC to a TextSignal as annotations

In [6]:
#### Example of an annotation function that adds annotations to a Signal
#### It adds NERC annotations to the TextSignal and returns a list of entities detected

def add_ner_annotation(signal: TextSignal):
    processor_name = "spaCy"
    utterance = ''.join(signal.seq)

    doc = nlp(utterance)

    offsets, tokens = zip(*[(Index(signal.id, token.idx, token.idx + len(token)), Token.for_string(token.text))
                            for token in doc])

    ents = [NER.for_string(ent.label_) for ent in doc.ents]
    entity_list = [ent.text for ent in doc.ents]
    segments = [token.ruler for token in tokens if token.value in entity_list]

    annotations = [Annotation(AnnotationType.TOKEN.name.lower(), token, processor_name, int(time.time()))
                   for token in tokens]
    ner_annotations = [Annotation(AnnotationType.NER.name.lower(), ent, processor_name, int(time.time()))
                       for ent in ents]

    signal.mentions.extend([Mention(str(uuid.uuid4()), [offset], [annotation])
                            for offset, annotation in zip(offsets, annotations)])
    signal.mentions.extend([Mention(str(uuid.uuid4()), [segment], [annotation])
                            for segment, annotation in zip(segments, ner_annotations)])
    print(entity_list)
    return entity_list

### 3.3 Creating a capsule from the interaction with a triple and perspective

The next function creates a capsule using the Scenario and Signal in combination with values for the perspectives, subject, predicate and object.

In [7]:
def seq_to_text (seq):
    text = ""
    for c in seq:
        text+=c
    return text



## 4 Starting the interaction

### 4.1 Creating a camera and nlp module

We create a camera and load the NLP model for spaCy.

In [8]:
### Link your camera
camera = cv2.VideoCapture(0)

### Load a language model in spaCy
nlp = spacy.load('en_core_web_sm')

### 4.2 Initialising a BRAIN in GraphDB

Before we start, we need to create an empty Brain or load an existing Brain. The next code assumes we have a repository in GraphDB with the name sandbox as a brain. By setting clear_all=True it is emptied and next loaded with the background ontologies.

In [10]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-10-27 09:32:39,448 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-10-27 09:32:39,449 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain
2021-10-27 09:32:43,455 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain
2021-10-27 09:32:43,459 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-10-27 09:32:44,208 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-10-27 09:32:46,103 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-10-27 09:32:46,105 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-10-27 09:32:46,106 -    DEBUG -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-10-27 09:32:46,108 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-10-27 09:32:46,196 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query
2021-10-27 09:32:46,216 -     INF

### 4.3 Starting the CHAT which creates a scenario and saves triples to the BRAIN

In [ ]:
##### First signals to get started
success, frame = camera.read()
imagepath = ""
if success:
    imagepath = imagefolder + "/" + str(datetime.now().microsecond) + ".png"
    cv2.imwrite(imagepath, frame)

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + human + "?"
print(agent + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(human + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    # @TODO
    ### Apply some processing to the textSignal and add annotations
    entityText = add_ner_annotation(textSignal)
    scenario.append_signal(textSignal)
    ## Post triples to the brain:

    subj, pred, obj = ttt.getTriplesFromEntities(entityText, textSignal.id)

    response = {}
    if not subj=="":
        print('Subject:', subj, 'Predicate:', pred, 'Object:', obj)
        perspective = {"certainty": 1, "polarity": 1, "sentiment": 1}
        capsule = c_util.scenario_utterance_to_capsule(scenario, textSignal, human, perspective, subj, pred, obj)
        #print('Capsule:', capsule)
        response = my_brain.update(capsule, reason_types=True)
        #print(thoughts)
        
    if success:
        imageSignal = d_util.create_image_signal(scenario, imagepath)
        # @TODO
        ### Apply some processing to the imageSignal and add annotations
        ### when done

        scenario.append_signal(imageSignal)

    # Create the response from the system and store this as a new signal
    utterance = ttt.getTextFromTriples(response)
    if not utterance:
        if not entityText:
            utterance = "Any gossip" + '\n'
        else:
            utterance = "So you what do you want to talk about " + entityText[0] + '\n'

    response = utterance[::-1]
    print(agent + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')

    success, frame = camera.read()
    if success:
        imagepath = imagefolder + "/" + str(datetime.now().microsecond) + ".png"
        cv2.imwrite(imagepath, frame)

Leolani2: Hi there. Who are you Stranger?



 Hi there


Stranger: Hi there
[]
Leolani2: Any gossip




 No news. I am Peter.


['Peter']
Subject: Peter Predicate: denotedBy Object: 640724cf-9907-4c67-a054-05ec054dfa9a
2021-10-27 12:34:20,077 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-10-27 12:34:20,142 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Posting query
2021-10-27 12:34:20,301 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: peter_denotedby_640724cf-9907-4c67-a054-05ec054dfa9a [person_->_object])
2021-10-27 12:34:20,303 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-10-27 12:34:20,343 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-10-27 12:34:20,389 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-10-27 12:34:20,396 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-10-27 12:34:20,397 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-10-27 12:34:20,446 -    DEBUG -  cltl.brain.

### After we stopped the interaction, we set the end time of the scenarion

In [ ]:
scenario.scenario.end = datetime.now().microsecond

### 4.4 Saving the Scenario

In [6]:
scenarioStorage.save_scenario(scenario)

## End of notebook